In [ ]:
from langchain_voyageai import VoyageAIEmbeddings
import os
import boto3
from urllib.parse import urlparse
from pinecone import Pinecone, ServerlessSpec
from langchain_openai import ChatOpenAI
import openai
from langchain.chains import LLMChain, RetrievalQA
import time
import re
from langchain_pinecone import PineconeVectorStore
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage
from IPython.display import Markdown, display

os.environ["VOYAGE_AI_API_KEY"] = "<VOYAGEAI_API_KEY>"
os.environ["PINECONE_API_KEY"] = "<PINECONE_API_KEY>"

model_name = "voyage-large-2"  
embeddings = VoyageAIEmbeddings(
    model=model_name,  
    voyage_api_key=os.environ["VOYAGE_AI_API_KEY"]
)

In [ ]:
# Function to generate pre-signed URL
def generate_presigned_url(s3_uri):
    # Parse the S3 URI
    parsed_url = urlparse(s3_uri)
    bucket_name = parsed_url.netloc
    object_key = parsed_url.path.lstrip('/')
    
    # Create an S3 client
    s3_client = boto3.client('s3')
    
    # Generate a pre-signed URL for the S3 object
    presigned_url = s3_client.generate_presigned_url(
        'get_object',
        Params={'Bucket': bucket_name, 'Key': object_key},
        ExpiresIn=3600  # URL expiration time in seconds
    )
    return presigned_url

# Function to retrieve documents, generate URLs, and format the response
def retrieve_and_format_response(query, retriever, llm):
    docs = retriever.get_relevant_documents(query)
    
    formatted_docs = []
    for doc in docs:
        content_data = doc.page_content
        s3_uri = doc.metadata['id']
        s3_gen_url = generate_presigned_url(s3_uri)
        formatted_doc = f"{content_data}\n\n[More Info]({s3_gen_url})"
        formatted_docs.append(formatted_doc)
    
    combined_content = "\n\n".join(formatted_docs)
    # print(combined_content)
    
    # Create a prompt for the LLM to generate an explanation based on the retrieved content
    prompt = f"Instruction: Based on the following information, provide a summarized & concise explanation using a couple of sentences. \
               Only respond with the information relevant to the user query {query}, if there are none, make sure you say 'I don't know, I did not find the relevant data in the knowledge base.' \
               In the event that there's relevant info, make sure to attach the download button at the very end: \n\n[More Info]({s3_gen_url}) \
               Context: {combined_content}"
    
    # Create the messages for the LLM input
    messages = [HumanMessage(content=prompt)]
    
    # Generate the response using the LLM
    response = llm(messages=messages)
    return {"answer": response.content}

In [ ]:
# PINECONE
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

index_name = "healthai-vector-embedding"

# Retriever
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

# Initialize LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key="<OPENAI_API_KEY")
# Initialize Memory
memory = ConversationBufferMemory()

In [ ]:
query1 = "What is Cyclacillin?"
query2 = "What drugs are being used to treat HIV?"
docs = docsearch.as_retriever().get_relevant_documents(query2)
docs

In [ ]:
# Example usage
response = retrieve_and_format_response(query1, docsearch.as_retriever(), llm=llm)

# Display the formatted response as Markdown
display(Markdown(response["answer"]))